In [5]:
from helper import DataPreprocessing
from ipywidgets import interact, IntSlider
from pyts.image import MarkovTransitionField
from pyts.approximation import PiecewiseAggregateApproximation as PAA
import matplotlib.pyplot as plt
import numpy as np

In [6]:
@interact(kind = ["train","full","test"])
def choose_kind_data(kind,n_bins = IntSlider(min=2, max = 200, step=1, value=10), strategy = ["quantile","uniform","normal"]):
    #load data
    data = DataPreprocessing(kind)
    
    #load method
    MTF = MarkovTransitionField(n_bins=n_bins,strategy=strategy)
    mt = "Markov Transition Field"
    method = "MTF"
    image_size = data.image_size
    paa = PAA(window_size=None,output_size=image_size)
    
    if kind in ["train", "full"]:
        if kind == "train":
            bearing_number =  [11,12,21,22,31,32]
        else:
            bearing_number =  [13,14,15,16,17,23,24,25,26,27,33]
        @interact(bearing = bearing_number)
        def load_data_train(bearing):
            
            #load data
            N = data.n_samples(kind,bearing)
            eol = data.eol(bearing)
            time_stamp = np.linspace(0,eol,N)
            norm = data.norm(bearing)
            w_size = data.window_size_stft
            sr = data.sr
            hor_acc = norm[0]
            ver_acc = norm[1]
            
            @interact(label = IntSlider(min=0,max=eol-90,step=10))
            def visualize_MTF (label):

                #scale data with label and step
                s_step = int(N - label * sr / 100  - w_size)
                time_stamp_w = time_stamp[s_step:s_step+w_size]
                time_stamp_w_real = np.linspace((eol-label)-90,(eol-label)+0.1,w_size)
                hor_acc_w = hor_acc[s_step:s_step+w_size]
                ver_acc_w = ver_acc[s_step:s_step+w_size]
                t_step = time_stamp_w_real.min()

                #visualize
                fig, ((ax1, ax2, ax3),(ax4, ax5, ax6)) = plt.subplots(nrows = 2, ncols = 3,figsize = (25,10))
                fig.suptitle("{} data, bearing {}, EOL {}s, {} samples\nMethod {}".format(kind,bearing,eol,N,mt))

                #axes 1:
                ax1.plot(time_stamp,hor_acc)
                ax1.plot(time_stamp_w,hor_acc_w)
                ax1.legend(["horizontal","window"],loc="upper left")
                ax1.set_ylabel("Normalized value")
                ax1.set_xlabel("Timestamp")
                ax1.set_title("Horizontal Acceleromete, time step {}".format(int(t_step)))

                #axes 2:
                ax2.plot(time_stamp_w_real,hor_acc_w,"darkorange")
                ax2.legend(["horizontal window"],loc = "upper left")
                ax2.set_ylabel("Normalized value")
                ax2.set_xlabel("Timestamp window")
                ax2.set_title("Horizontal Acceleromete Windowm, label {}".format(label))
                ax2.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax2.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])

                #axes 3:(paa.transform(hor_acc_w.reshape(1,-1)))
                mtf_hor = MTF.fit_transform
                image_mtf_hor = ax3.imshow(mtf_hor.transpose(1,2,0))
                image_mtf_hor.set_clim(vmin=0,vmax=1)
                plt.colorbar(image_mtf_hor,ax=ax3)
                ax3.set_title("{}, size {}".format(method,mtf_hor.shape))

                #axes 4:
                ax4.plot(time_stamp,ver_acc)
                ax4.plot(time_stamp_w,ver_acc_w)
                ax4.legend(["vertikal","window"],loc="upper left")
                ax4.set_ylabel("Normalized value")
                ax4.set_xlabel("Timestamp")
                ax4.set_title("Vertikal Acceleromete,time step {}".format(int(t_step)))
                
                #axes 5:
                ax5.plot(time_stamp_w_real,ver_acc_w,"darkorange")
                ax5.legend(["vertikal window"],loc = "upper left")
                ax5.set_ylabel("Normalized value")
                ax5.set_xlabel("Timestamp window")
                ax5.set_title("Horizontal Acceleromete Window label {}".format(label))
                ax5.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax5.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])

                #axes 6:
                mtf_ver = MTF.fit_transform(paa.transform(ver_acc_w.reshape(1,-1)))
                image_mtf_ver = ax6.imshow(mtf_ver.transpose(1,2,0))
                image_mtf_ver.set_clim(vmin=0,vmax=1)
                plt.colorbar(image_mtf_ver,ax=ax6)
                ax6.set_title("{}, size {}".format(method,mtf_hor.shape))

                print("Done")
                
    elif kind == "test":
        @interact(bearing = [13,14,15,16,17,23,24,25,26,27,33])
        def load_data_test(bearing):
            
            #load data
            N = data.n_samples(kind,bearing)
            last_cycle = data.last_cycle(bearing)
            eol = data.eol(bearing)
            rul = data.rul(bearing)
            time_stamp = np.linspace(0,last_cycle,N)
            norm = data.norm(bearing)
            w_size = data.window_size_stft
            sr = data.sr
            hor_acc = norm[0]
            ver_acc = norm[1]
            
            @interact(label = IntSlider(min=rul,max=rul+last_cycle-90,step=10))
            def visualize_MTF (label):

                #scale data with label and step
                s_step = int(N - (label-rul) * sr / 100  - w_size) 
                time_stamp_w = time_stamp[s_step:s_step+w_size]
                time_stamp_w_real = np.linspace((last_cycle-label+rul)+0,(last_cycle-label+rul)+0.1,sr)
                hor_acc_w = hor_acc[s_step:s_step+w_size]
                ver_acc_w = ver_acc[s_step:s_step+w_size]
                t_step = time_stamp_w_real.min()

                #visualize
                fig, ((ax1, ax2, ax3),(ax4, ax5, ax6)) = plt.subplots(nrows = 2, ncols = 3,figsize = (25,10))
                fig.suptitle("{} data, bearing {}, EOL {} s, RUL {} s, {} samples\nMethod {}".format(kind,bearing,eol,rul,N,mt))

                #axes 1:
                ax1.plot(time_stamp,hor_acc)
                ax1.plot(time_stamp_w,hor_acc_w)
                ax1.legend(["horizontal","window"],loc="upper left")
                ax1.set_ylabel("Normalized value")
                ax1.set_xlabel("Timestamp")
                ax1.set_title("Horizontal Acceleromete, step {}".format(int(t_step)))

                #axes 2:
                ax2.plot(time_stamp_w_real,hor_acc_w,c="darkorange")
                ax2.legend(["horizontal window"],loc = "upper left")
                ax2.set_ylabel("Normalized value")
                ax2.set_xlabel("Timestamp window")
                ax2.set_title("Horizontal Acceleromete Window, label {}".format(label))
                ax2.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax2.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])

                #axes 3:
                mtf_hor = MTF.fit_transform(paa.transform(hor_acc_w.reshape(1,-1)))
                image_mtf_hor = ax3.imshow(mtf_hor.transpose(1,2,0))
                image_mtf_hor.set_clim(vmin=0,vmax=1)
                plt.colorbar(image_mtf_hor,ax=ax3)
                ax3.set_title("{}, size {}".format(method,mtf_hor.shape))

                #axes 4:
                ax4.plot(time_stamp,ver_acc)
                ax4.plot(time_stamp_w,ver_acc_w)
                ax4.legend(["vertikal","window"],loc="upper left")
                ax4.set_ylabel("Normalized value")
                ax4.set_xlabel("Timestamp")
                ax4.set_title("Vertikal Acceleromete, step {}".format(int(t_step)))
                
                #axes 5:
                ax5.plot(time_stamp_w_real,ver_acc_w,c="darkorange")
                ax5.legend(["vertikal window"],loc = "upper left")
                ax5.set_ylabel("Normalized value")
                ax5.set_xlabel("Timestamp")
                ax5.set_title("Horizontal Acceleromete Window, label {}".format(label))
                ax5.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax5.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])

                #axes 6:
                mtf_ver = MTF.fit_transform(paa.transform(ver_acc_w.reshape(1,-1)))
                image_mtf_ver = ax6.imshow(mtf_ver.transpose(1,2,0))
                image_mtf_ver.set_clim(vmin=0,vmax=1)
                plt.colorbar(image_mtf_ver,ax=ax6)
                ax6.set_title("{}, size {}".format(method,mtf_hor.shape))

                print("Done")

interactive(children=(Dropdown(description='kind', options=('train', 'full', 'test'), value='train'), IntSlide…